In [17]:
import geopandas as gpd
import pandas as pd
import numpy as np
from programa_geovars import *

In [18]:
vars_postal_code = pd.read_csv("vars_censo_codigo_postal_def.csv")
cod_postales = gpd.read_file("codigos_postales_madrid/codigos_postales_madrid.shp")

In [100]:
total_tr = pd.read_csv('TOTAL_TRAIN.csv')

/home/alejandro.vaca/miniconda/envs/sentinel_env/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (54) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [36]:
cols_rellenar = total_tr.columns[total_tr.isna().sum()>20]

In [62]:
cols_rellenar

Index(['edad_media', 'p_poblacion_menor_de_18', 'p_poblacion_mayor_65',
       'media_personas_por_hogar', 'poblacion_municipio', 'poblacion_cp',
       'crecimiento_poblaconal_2015', 'renta_media_por_persona',
       'renta_media_por_hogar', 'p_suelo_urbano', 'p_agropecuario',
       'p_cultural', 'p_deportivo', 'p_industrial', 'p_oficina', 'p_otros',
       'p_publico', 'p_residencial', 'p_retail', 'p_salud', 'p_solteros',
       'p_casados', 'p_viudos', 'p_separados', 'p_divorciados',
       'p_extranjeros', 'p_hogares_unipersonales', 'p_ocup_agropecuaria',
       'p_ocup_pesca', 'p_ocup_mineria', 'p_ocup_manufactura',
       'p_ocup_dirstr_gas_elec_agua', 'p_ocup_construccion',
       'p_ocup_taller_automotriz', 'p_ocup_hosteleria', 'p_ocup_transporte',
       'p_ocup_financiera', 'p_ocup_inmobiliarias', 'p_ocup_adm_publica',
       'p_ocup_educacion', 'p_ocup_medicina', 'p_ocup_otros',
       'p_casas_con_servicio_domestico', 'prop_masculinidad',
       'prop_reemplazamiento', 'pr

In [51]:
#total_tr[total_tr[cols_rellenar].isna()].head()#[cols_rellenar].head()
#filtered_df = df[df[['name', 'country', 'region']].notnull().all(1)]
#total_tr[total_tr[cols_rellenar].isna()]

In [67]:
#index = total_tr[cols_rellenar].index[total_tr[cols_rellenar].isna()]
inds = pd.isnull(total_tr[cols_rellenar]).any(1)

In [68]:
sum(inds)

66

In [70]:
#total_tr.loc[inds, cols_rellenar]

In [21]:
[code for code in total_tr.CODIGO_POSTAL.unique() if code not in vars_postal_code.CODIGO_POSTAL.unique()]

[28907.0]

In [22]:
vars_postal_code[vars_postal_code.CODIGO_POSTAL == 28903]['CODIGO_POSTAL']

219    28903
Name: CODIGO_POSTAL, dtype: int64

In [24]:
total_tr.loc[total_tr.CODIGO_POSTAL == 28907.0,'CODIGO_POSTAL'] = 28903

In [25]:
merged = pd.merge(total_tr, vars_postal_code, on='CODIGO_POSTAL', how='left')

In [27]:
#merged.isna().sum()

In [31]:
total_tr[total_tr.p_divorciados.isna()]['CODIGO_POSTAL'].unique()

array([28903., 28524.])

In [35]:
merged[merged.edad_media_y.isna()]['CODIGO_POSTAL'].unique()

array([], dtype=float64)

In [101]:
df = total_tr.copy()

In [121]:
cols_rellenar = df.columns[df.isna().sum()>20]
#especiales = ['p_solteros','p_casados', 'p_viudos', 'p_separados', 'p_divorciados']
#cols_rellenar1 = [col for col in cols_rellenar if col not in especiales]
inds_1 = (pd.isnull(df[cols_rellenar]).any(1)) & (df.CODIGO_POSTAL == 28907)

In [117]:
#total_tr.loc[inds_1, cols_rellenar1] = vars_postal_code.loc[vars_postal_code.CODIGO_POSTAL == 28903, cols_rellenar1]

In [122]:
#for col in cols_rellenar:
total_tr.loc[inds_1, cols_rellenar] = vars_postal_code.loc[vars_postal_code.CODIGO_POSTAL == 28903, cols_rellenar].values

In [124]:
#total_tr.isna().sum()

In [ ]:
#total_tr[total_tr[especiales].isna()]['CODIGO_POSTAL']

In [125]:
def fill_cods_nas(df):
    cols_rellenar = df.columns[df.isna().sum()>20]
    especiales = ['p_solteros','p_casados', 'p_viudos', 'p_separados', 'p_divorciados']
    #cols_rellenar1 = [col for col in cols_rellenar if col not in especiales]
    inds = (pd.isnull(df[cols_rellenar]).any(1)) & (df.CODIGO_POSTAL == 28907)
    df.loc[inds, cols_rellenar] = vars_postal_code.loc[vars_postal_code.CODIGO_POSTAL == 28903, cols_rellenar].values
    df.loc[df.CODIGO_POSTAL == 28524, especiales] = vars_postal_code.loc[vars_postal_code.CODIGO_POSTAL == 28522, especiales].values
    return df

In [126]:
total_tr = pd.read_csv('TOTAL_TRAIN.csv')

In [127]:
total_tr = fill_cods_nas(total_tr)

In [132]:
total_tr['renta_media_por_hogar'] = [total_tr.loc[i, 'renta_media_por_hogar'].replace(',', '') for i in range(total_tr.shape[0])]

In [133]:
total_tr['renta_media_por_hogar'] = total_tr['renta_media_por_hogar'].astype('float')

In [134]:
total_tr.renta_media_por_hogar

0         36786.0
1         36786.0
2         36786.0
3         36786.0
4         36786.0
           ...   
103225    55269.0
103226    31763.0
103227    41252.0
103228    41252.0
103229    41252.0
Name: renta_media_por_hogar, Length: 103230, dtype: float64